In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from tika import parser # PDF Parser
import pandas as pd
import requests
import time
import re

In [ ]:
URL = 'https://www.iitp.kr/kr/1/knowledge/periodicalViewA.it'
payload = {'searClassCode': 'B_ITA_01'
,'masterCode': 'publication'
,'identifier': 1155}
pages = [x for x in range(1036, 1156)]

In [ ]:
target = []
for x in pages:
    payload['identifier'] = x
    r = requests.get(URL, params = payload)
    target.append(r.url)

In [ ]:
# Disable Chrome's PDF Viewer
download_dir = '../IITP ICT 신기술 동향/'
prefs = {
"download.default_directory": download_dir, #Change default directory for downloads
"download.prompt_for_download": False, #To auto download the file
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
}

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless')
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome('./chromedriver', chrome_options=chrome_options)

In [ ]:
for link in target:
    driver.get(link)
    time.sleep(2.5)
    try:
        driver.find_element_by_xpath('//*[@id="conArea"]/div[1]/div[1]/div/div[3]/ul/li/a').click()
    except:
        print('Page not found 404 error')

In [ ]:
driver.close()

In [ ]:
report = parser.from_file('../../Downloads/file1014888888178779151-1854.pdf')

In [ ]:
def preprocessing(text):
    # 불필요 Text 정리
    
    text = text.split('ICT 신기술 최신')[1]
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', '', text)
    text = re.sub('\r', '', text)
    text = re.sub('III', '', text)
    text = re.sub('(http|https):\/\/(\w+:{0,1}\w*@)?(\S+)(:[0-9]+)?(\/|\/([\w#!:.?+=&%@!\-\/]))?', '', text)
    text = re.sub('(www)(\S+)(:[0-9]+)?(\/|\/([\w#!:.?+=&%@!\-\/]))?', '', text)
    text = re.sub('n', '', text)
    text = re.sub('Ⅲ', '', text)
    text = re.sub('[▸|◾|☎]', '', text)
    text = re.sub('d{3}-\d{4}-\d{4}', '', text)
    text = re.sub('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', '', text)
    text = re.sub('\\\\\\\\', '', text)
    text = re.sub('\\\\' , '', text)
   # text = re.sub('[\[.*\]]', '', text)
    text = re.sub('^[\[그림.*]]$', '', text)
    text = text.split('사업책임자')[0]
    
    return text

In [ ]:
res = preprocessing(str(report))
res